In [2]:
from doped.generation import DefectsGenerator
from monty.serialization import loadfn, dumpfn
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPRelaxSet, MPMetalRelaxSet
from pymatgen.io.vasp import Poscar

In [4]:
Al_prim = loadfn('../../data/matching_AlCr_ox.json')

In [5]:
for key in Al_prim.keys():
    structure = Al_prim[key]['structure']
    mp_relax_set = MPRelaxSet(structure)
    mp_relax_set.write_input(f"{Al_prim[key]['formula']}/relax_prim_{Al_prim[key]['formula']}")

In [7]:
poscar = Poscar.from_file("Al2(SO4)3/relax_prim_Al2(SO4)3/CONTCAR")
relaxed_structure = poscar.structure

In [8]:
defect_gen = DefectsGenerator(
    relaxed_structure, 
    extrinsic=["Cr"],
    interstitial_gen_kwargs=False
)

Generating DefectEntry objects: 100.0%|████████████████████████████████████████████████████████████| [02:35,   1.55s/it]

Vacancies                 Guessed Charges           Conv. Cell Coords    Wyckoff
------------------------  ------------------------  -------------------  ---------
v_Al_C3_O1.89             [+1,0,-1,-2,-3]           [0.000,0.000,0.143]  6c
v_Al_C3_O1.90             [+1,0,-1,-2,-3]           [0.000,0.000,0.352]  6c
v_S                       [+1,0,-1,-2,-3,-4,-5,-6]  [0.289,0.002,0.251]  18f
v_O_C1_S1.47              [+2,+1,0,-1]              [0.173,0.195,0.090]  18f
v_O_C1_S1.48Al1.89        [+2,+1,0,-1]              [0.013,0.200,0.192]  18f
v_O_C1_S1.48Al1.90O2.38a  [+2,+1,0,-1]              [0.219,0.073,0.302]  18f
v_O_C1_S1.48Al1.90O2.38b  [+2,+1,0,-1]              [0.144,0.218,0.402]  18f

Substitutions              Guessed Charges              Conv. Cell Coords    Wyckoff
-------------------------  ---------------------------  -------------------  ---------
Al_S                       [0,-1,-2,-3]                 [0.289,0.002,0.251]  18f
Al_O_C1_S1.47              [+5,+4,+3,+2,+1,0]

In [9]:
defect_gen.defects = {"substitutions": [d for d in defect_gen.defects["substitutions"] if d.name == "Cr_Al"]}
defect_gen.defect_entries = {k: v for k, v in defect_gen.defect_entries.items() if "Cr_Al" in k and k[-1] == "0"}
defect_gen

DefectsGenerator for input composition Al2(SO4)3, space group R-3 with 2 defect entries created.
---------------------------------------------------------
Substitutions    Guessed Charges    Conv. Cell Coords    Wyckoff
---------------  -----------------  -------------------  ---------
Cr_Al_C3_O1.89   [0]                [0.000,0.000,0.143]  6c
Cr_Al_C3_O1.90   [0]                [0.000,0.000,0.352]  6c

The number in the Wyckoff label is the site multiplicity/degeneracy of that defect in the conventional ('conv.') unit cell, which comprises 6 formula unit(s) of Al2(SO4)3.

In [12]:
from doped.vasp import DefectsSet

LDAU_PARAMS = {
  "LDAU": True,
    "LDAUJ": {
        "F": {
            "Co": 0,
            "Cr": 0,
            "Fe": 0,
            "Mn": 0,
            "Mo": 0,
            "Ni": 0,
            "V": 0,
            "W": 0
        },
        "O": {
            "Co": 0,
            "Cr": 0,
            "Fe": 0,
            "Mn": 0,
            "Mo": 0,
            "Ni": 0,
            "V": 0,
            "W": 0
        }
    },
    "LDAUL": {
        "F": {
            "Co": 2,
            "Cr": 2,
            "Fe": 2,
            "Mn": 2,
            "Mo": 2,
            "Ni": 2,
            "V": 2,
            "W": 2
        },
        "O": {
            "Co": 2,
            "Cr": 2,
            "Fe": 2,
            "Mn": 2,
            "Mo": 2,
            "Ni": 2,
            "V": 2,
            "W": 2
        }
    },
    "LDAUTYPE": 2,
    "LDAUU": {
        "F": {
            "Co": 3.32,
            "Cr": 3.7,
            "Fe": 5.3,
            "Mn": 3.9,
            "Mo": 4.38,
            "Ni": 6.2,
            "V": 3.25,
            "W": 6.2
        },
        "O": {
            "Co": 3.32,
            "Cr": 3.7,
            "Fe": 5.3,
            "Mn": 3.9,
            "Mo": 4.38,
            "Ni": 6.2,
            "V": 3.25,
            "W": 6.2
        }
    },
    "MAGMOM": {
        "Ce": 5,
        "Ce3+": 1,
        "Co": 0.6,
        "Co3+": 0.6,
        "Co4+": 1,
        "Cr": 5,
        "Dy3+": 5,
        "Er3+": 3,
        "Eu": 10,
        "Eu2+": 7,
        "Eu3+": 6,
        "Fe": 5,
        "Gd3+": 7,
        "Ho3+": 4,
        "La3+": 0.6,
        "Lu3+": 0.6,
        "Mn": 5,
        "Mn3+": 4,
        "Mn4+": 3,
        "Mo": 5,
        "Nd3+": 3,
        "Ni": 5,
        "Pm3+": 4,
        "Pr3+": 2,
        "Sm3+": 5,
        "Tb3+": 6,
        "Tm3+": 2,
        "V": 5,
        "W": 5,
        "Yb3+": 1
    }
}

defect_set = DefectsSet(
    defect_gen,
    user_incar_settings={"ENCUT": 520, "LHFCALC": None, "HFSCREEN": None, "AEXX": None, "GGE":"PBE", **LDAU_PARAMS},
    soc=False,
    user_potcar_functional="PBE",
)
defect_set.write_files("doped-defs-Al2SO43", poscar=True)

vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
Generating and writing input files: 100%|█████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.36it/s]
